In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.forecasting.theta import ThetaModel
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import train_test_split
import warnings
import matplotlib.pyplot as plt
from darts import TimeSeries
from darts.models import Croston

warnings.filterwarnings("ignore")

Importing plotly failed. Interactive plots will not work.


In [2]:
# Load the data
file_path = '100_SKUS.csv'
data = pd.read_csv(file_path)
data_long = pd.melt(data, id_vars=['material'], var_name='date', value_name='sales')
data_long['date'] = pd.to_datetime(data_long['date'], format='%m/%d/%Y')
data_long = data_long.sort_values(by=['material', 'date'])

In [3]:
data_long

,material,date,sales
29,sku1009,2019-04-01,88
78,sku1009,2019-05-01,205
127,sku1009,2019-06-01,363
176,sku1009,2019-07-01,403
225,sku1009,2019-08-01,342
...,...,...,...
2575,sku949,2023-08-01,42
2624,sku949,2023-09-01,10
2673,sku949,2023-10-01,10
2722,sku949,2023-11-01,36


In [4]:
def create_features(df, label=None):
    df['month'] = df.index.month
    df['year'] = df.index.year
    X = df[['month', 'year']]
    if label:
        y = df[label]
        return X, y
    return X

def evaluate_forecasts(actual, predicted):
    mape = mean_absolute_percentage_error(actual, predicted) * 100
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    accuracy = 100 - mape
    return accuracy, rmse

In [46]:
results = []

# Function to train and test models
def train_and_test_models(train, test, material, month):
    # Convert train and test data to TimeSeries
    train_series = TimeSeries.from_series(train, fill_missing_dates=True, freq='MS')
    test_series = TimeSeries.from_series(test, fill_missing_dates=True, freq='MS')
   
    models = {
        'ARIMA': ARIMA(train, order=(2, 1, 2)),  # Replace with appropriate parameters
        'SARIMA': SARIMAX(train, order=(2, 1, 2), seasonal_order=(1, 1, 1, 12)),  # Replace with appropriate parameters
        'Theta': ThetaModel(train),
        'Croston': Croston(version="optimized")
    }

    # Train and forecast
    forecasts = {}
    for name, model in models.items():
        try:
            if name in ['ARIMA', 'SARIMA']:
                model_fit = model.fit()
                forecast = model_fit.forecast(steps=2)[-1]
            elif name in ['Theta']:
                model_fit = model.fit()
                forecast = model_fit.forecast(steps=2).iloc[-1]
            else:
                model.fit(train_series)
                forecast = model.predict(2).values().flatten()[-1]
                
            if forecast < 0:
                forecast = 0  # Set negative forecasts to zero
    
            forecast = np.array([forecast])
                
            forecasts[name] = forecast
        except Exception as e:
            print(f"Error fitting model {name} for material {material} in month {month}: {e}")
            forecasts[name] = np.full(len(test), np.nan)

    # Evaluate models
    accuracies_rmse = {name: evaluate_forecasts(test, forecast) for name, forecast in forecasts.items() if not np.isnan(forecast).all()}

    # Save results for all models
    for name, (accuracy, rmse) in accuracies_rmse.items():
        results.append({
            'material': material,
            'month': month,
            'model': name,
            'accuracy': accuracy,
            'rmse': rmse,
            'actual': test.values,
            'predicted': forecasts[name]
        })

In [47]:
# Train and test with rolling windows
materials = data_long['material'].unique()
for material in materials:
    sku_data = data_long[data_long['material'] == material].set_index('date')['sales']
    dates = pd.date_range(start='2023-07-01', end='2023-12-01', freq='MS')

    for date in dates:
        train = sku_data[:date - pd.DateOffset(months=2)]
        test = sku_data[date:date]
        if len(test) > 0:
            train_and_test_models(train, test, material, date.strftime('%Y-%m'))

# Create DataFrame for results
results_df = pd.DataFrame(results)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            7     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.79163D+00    |proj g|=  1.91601D-01

At iterate    5    f=  4.70901D+00    |proj g|=  3.24681D-02

At iterate   10    f=  4.67959D+00    |proj g|=  3.98220D-03

At iterate   15    f=  4.67944D+00    |proj g|=  4.92748D-03


 This problem is unconstrained.



At iterate   20    f=  4.67885D+00    |proj g|=  1.34295D-03

At iterate   25    f=  4.67858D+00    |proj g|=  1.13245D-02

At iterate   30    f=  4.66251D+00    |proj g|=  1.50858D-02

At iterate   35    f=  4.65853D+00    |proj g|=  1.83999D-02

At iterate   40    f=  4.65596D+00    |proj g|=  1.52454D-03

At iterate   45    f=  4.65519D+00    |proj g|=  2.18458D-03

At iterate   50    f=  4.65508D+00    |proj g|=  5.51906D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    7     50     58      1     0     0   5.519D-04   4.655D+00
  F =   4.6550846802685415     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT       

 This problem is unconstrained.



At iterate   25    f=  4.68954D+00    |proj g|=  8.24991D-03

At iterate   30    f=  4.68831D+00    |proj g|=  1.54134D-03

At iterate   35    f=  4.68813D+00    |proj g|=  1.42950D-03

At iterate   40    f=  4.68804D+00    |proj g|=  1.79615D-03

At iterate   45    f=  4.68802D+00    |proj g|=  1.45752D-04

At iterate   50    f=  4.68802D+00    |proj g|=  3.03174D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    7     50     56      1     0     0   3.032D-05   4.688D+00
  F =   4.6880212880587271     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 
RUNNING THE L-BFGS-B CODE

           * * *

Machin

...


At iterate    5    f=  5.21506D+00    |proj g|=  3.36267D-02

At iterate   10    f=  5.20639D+00    |proj g|=  2.71703D-03

At iterate   15    f=  5.20511D+00    |proj g|=  5.38568D-03

At iterate   20    f=  5.20500D+00    |proj g|=  5.61245D-04

At iterate   25    f=  5.20409D+00    |proj g|=  9.81463D-04

At iterate   30    f=  5.20243D+00    |proj g|=  1.46760D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    7     32     38      1     0     0   8.406D-06   5.202D+00
  F =   5.2024344320051945     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
RUNNING THE L-BFGS-B CODE

           * * *

Machin

 This problem is unconstrained.



At iterate   15    f=  5.22631D+00    |proj g|=  7.75409D-03

At iterate   20    f=  5.22617D+00    |proj g|=  7.41902D-04

At iterate   25    f=  5.22493D+00    |proj g|=  5.74542D-03

At iterate   30    f=  5.22350D+00    |proj g|=  2.56291D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    7     32     35      1     0     0   1.121D-05   5.223D+00
  F =   5.2234994108444077     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            7     M =           10

At X0         0 variables are exactly at the bounds

At iter

 This problem is unconstrained.



At iterate    5    f=  5.25648D+00    |proj g|=  3.83912D-02

At iterate   10    f=  5.24774D+00    |proj g|=  2.08500D-03

At iterate   15    f=  5.24645D+00    |proj g|=  5.71973D-03

At iterate   20    f=  5.24605D+00    |proj g|=  1.45163D-02

At iterate   25    f=  5.24357D+00    |proj g|=  1.80650D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    7     29     32      1     0     0   4.481D-06   5.244D+00
  F =   5.2435645539464693     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


In [48]:
results_df

,material,month,model,accuracy,rmse,actual,predicted
0,sku1009,2023-07,ARIMA,60.691498,42.060097,[107],[64.93990294024036]
1,sku1009,2023-07,SARIMA,-29.901977,138.995115,[107],[245.9951152888486]
2,sku1009,2023-07,Theta,-178.380133,297.866743,[107],[404.8667425140196]
3,sku1009,2023-07,Croston,78.482284,23.023956,[107],[83.97604]
4,sku1009,2023-08,ARIMA,79.707894,15.219079,[75],[59.78092059951472]
...,...,...,...,...,...,...,...
1170,sku949,2023-11,Croston,-63.520474,58.867371,[36],[94.86737]
1171,sku949,2023-12,ARIMA,-1170.571257,152.468551,[12],[164.46855082576064]
1172,sku949,2023-12,SARIMA,-603.293357,84.395203,[12],[96.39520280253603]
1173,sku949,2023-12,Theta,-349.927910,53.991349,[12],[65.99134922809043]


In [49]:
# Compute average RMSE and accuracy for each model for each SKU
average_metrics_df = results_df.groupby(['material', 'model']).agg(
    average_rmse=('rmse', 'mean'),
    average_accuracy=('accuracy', 'mean')
).reset_index()
average_metrics_df

,material,model,average_rmse,average_accuracy
0,sku1009,ARIMA,17.181394,79.661065
1,sku1009,Croston,17.936681,75.855450
2,sku1009,SARIMA,60.258731,28.989200
3,sku1009,Theta,115.457147,-33.751326
4,sku1011,ARIMA,32.073076,10.784425
...,...,...,...,...
191,sku947,Theta,111.502155,36.458203
192,sku949,ARIMA,135.430047,-779.257020
193,sku949,Croston,70.204672,-396.249571
194,sku949,SARIMA,43.867866,-88.900853


In [50]:
# Identify the best model for each SKU based on the lowest average RMSE
best_models_df = average_metrics_df.loc[average_metrics_df.groupby('material')['average_rmse'].idxmin()]
best_models_df

,material,model,average_rmse,average_accuracy
0,sku1009,ARIMA,17.181394,7.966106e+01
5,sku1011,Croston,19.772200,3.602575e+01
11,sku1080,Theta,53.532801,5.756803e+01
13,sku1081,Croston,23.395326,4.610118e+01
16,sku1119,ARIMA,92.507433,-1.346880e+02
20,sku1123,ARIMA,103.840768,-1.716138e+00
25,sku1127,Croston,37.976480,3.833205e+01
28,sku1129,ARIMA,23.491930,2.187025e+01
33,sku1174,Croston,31.955231,5.395887e+01
37,sku1176,Croston,22.119630,4.652787e+01


In [51]:
# Save the results to CSV files
results_df.to_csv("result.csv")
average_metrics_df.to_csv("average_rmse.csv")
best_models_df.to_csv("best_models.csv")

In [58]:
best_model_forecasts = []

for index, row in best_models_df.iterrows():
    material = row['material']
    best_model_name = row['model']

    sku_data = data_long[data_long['material'] == material].set_index('date')['sales']
    train = sku_data[:'2023-12-31']
    
    if best_model_name == 'ARIMA':
        model = ARIMA(train, order=(2, 1, 2))  
    elif best_model_name == 'SARIMA':
        model = SARIMAX(train, order=(2, 1, 2), seasonal_order=(1, 1, 1, 12)) 
    elif best_model_name == 'Theta':
        model = ThetaModel(train)
    elif best_model_name == 'Croston':
        train_series = TimeSeries.from_series(train, fill_missing_dates=True, freq='MS')
        model = Croston(version="optimized")

    # Forecasting for February 2024
    if best_model_name in ['ARIMA', 'SARIMA', 'Theta']:
        model_fit = model.fit()
        forecast = model_fit.forecast(steps=2)
        forecast_value = forecast.iloc[-1]  # February 2024
    elif best_model_name == 'Croston':
        model.fit(train_series)
        forecast = model.predict(2).values().flatten()
        forecast_value = forecast[-1]  # February 2024
    else:
        future_dates = pd.date_range(start='2023-12-31', periods=2, freq='MS')
        X_future = create_features(pd.DataFrame(index=future_dates))
        forecast = model.predict(X_future)
        forecast_value = forecast[-1]  # February 2024

    best_model_forecasts.append({
        'material': material,
        'best_model': best_model_name,
        'forecast_value': max(0, forecast_value)  # no negative forecast
    })

# Create DataFrame for best model forecasts
best_model_forecasts_df = pd.DataFrame(best_model_forecasts)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            7     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.19813D+00    |proj g|=  2.04469D-01

At iterate    5    f=  5.11642D+00    |proj g|=  8.52852D-02

At iterate   10    f=  5.10137D+00    |proj g|=  5.88427D-03


 This problem is unconstrained.



At iterate   15    f=  5.10027D+00    |proj g|=  1.85106D-03

At iterate   20    f=  5.10008D+00    |proj g|=  3.00904D-03

At iterate   25    f=  5.09981D+00    |proj g|=  1.64698D-03

At iterate   30    f=  5.09869D+00    |proj g|=  7.56211D-03

At iterate   35    f=  5.09531D+00    |proj g|=  4.65587D-03

At iterate   40    f=  5.09358D+00    |proj g|=  6.97730D-04

At iterate   45    f=  5.09353D+00    |proj g|=  3.13073D-03

At iterate   50    f=  5.09350D+00    |proj g|=  1.31692D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    7     50     62      1     0     0   1.317D-04   5.094D+00
  F =   5.09350319702

 This problem is unconstrained.



At iterate   20    f=  5.54341D+00    |proj g|=  1.69108D-02

At iterate   25    f=  5.52048D+00    |proj g|=  5.51574D-02

At iterate   30    f=  5.49266D+00    |proj g|=  3.97112D-02

At iterate   35    f=  5.48922D+00    |proj g|=  4.67959D-03

At iterate   40    f=  5.48882D+00    |proj g|=  1.43300D-03

At iterate   45    f=  5.48876D+00    |proj g|=  9.31155D-04

At iterate   50    f=  5.48875D+00    |proj g|=  2.51549D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    7     50     61      1     0     0   2.515D-04   5.489D+00
  F =   5.4887455250568484     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT       

...


At iterate   10    f=  6.63651D+00    |proj g|=  7.15693D-03

At iterate   15    f=  6.63481D+00    |proj g|=  4.10652D-03

At iterate   20    f=  6.63465D+00    |proj g|=  1.80734D-04

At iterate   25    f=  6.62935D+00    |proj g|=  2.43994D-02

At iterate   30    f=  6.62080D+00    |proj g|=  6.66027D-03

At iterate   35    f=  6.61950D+00    |proj g|=  1.85587D-03

At iterate   40    f=  6.59645D+00    |proj g|=  1.23583D-02

At iterate   45    f=  6.58574D+00    |proj g|=  1.46057D-02

At iterate   50    f=  6.57292D+00    |proj g|=  1.08881D-02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    7     50     72   

 This problem is unconstrained.



At iterate   15    f=  5.28040D+00    |proj g|=  8.79782D-04

At iterate   20    f=  5.28030D+00    |proj g|=  6.37444D-03

At iterate   25    f=  5.27687D+00    |proj g|=  3.32510D-02

At iterate   30    f=  5.25991D+00    |proj g|=  6.16133D-03

At iterate   35    f=  5.25873D+00    |proj g|=  1.59467D-03

At iterate   40    f=  5.25856D+00    |proj g|=  1.47134D-03

At iterate   45    f=  5.25853D+00    |proj g|=  7.50927D-04

At iterate   50    f=  5.25853D+00    |proj g|=  3.02664D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    7     50     55      1     0     0   3.027D-04   5.259D+00
  F =   5.25853086684

 This problem is unconstrained.



At iterate   20    f=  5.28680D+00    |proj g|=  9.64851D-04

At iterate   25    f=  5.28455D+00    |proj g|=  5.85404D-03

At iterate   30    f=  5.28317D+00    |proj g|=  9.33635D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    7     32     40      1     0     0   3.056D-06   5.283D+00
  F =   5.2831721867219050     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


In [59]:
# Save the forecast results to a CSV file
best_model_forecasts_df.to_csv("best_model_forecasts.csv", index=False)

print("Best Model Forecasts for February 2024:")
best_model_forecasts_df

Best Model Forecasts for February 2024:


,material,best_model,forecast_value
0,sku1009,ARIMA,85.810834
1,sku1011,Croston,49.689156
2,sku1080,Theta,94.294449
3,sku1081,Croston,56.284107
4,sku1119,ARIMA,144.027794
5,sku1123,ARIMA,157.719270
6,sku1127,Croston,81.559723
7,sku1129,ARIMA,33.733656
8,sku1174,Croston,109.060852
9,sku1176,Croston,60.537754
